# Import Libaries

In [6]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import urllib.request

from zipfile import ZipFile
from io import BytesIO
import os, zipfile

# Important Address URLS

In [7]:
Fla_csv = "https://opendata.bosa.be/download/best/openaddress-bevlg.zip"
Bru_csv = "https://opendata.bosa.be/download/best/openaddress-bebru.zip"
Wal_csv = "https://opendata.bosa.be/download/best/openaddress-bewal.zip"

# Download and Unzip files

In [8]:
csv_urls = [Fla_csv, Bru_csv, Wal_csv]
file_names= ["Fla_csv", "Bru_csv", "Wal_csv"]

# If address data folder not present in dir create it.
if not os.path.exists('address data'):
    os.makedirs('address data')
    
    # start downloading data from urls
    print('Beginning file download with urllib2...')

    for url, file_name in zip(csv_urls,file_names):
        urllib.request.urlretrieve(url, f'./address data/{file_name}.zip')
    
    # Unzip and remove move zip files
    dir_name = os.getcwd()+"\\address data"
    extension = ".zip"

    os.chdir(dir_name) # change directory from working dir to dir with files

    for item in os.listdir(dir_name): # loop through items in dir
        if item.endswith(extension): # check for ".zip" extension
            file_name = os.path.abspath(item) # get full path of files
            zip_ref = zipfile.ZipFile(file_name) # create zipfile object
            zip_ref.extractall(dir_name) # extract file to dir
            zip_ref.close() # close file
            os.remove(file_name) # delete zipped file
else:
    dir_name = os.getcwd()+"\\address data"
    os.chdir(dir_name)

# Generate Dataframe

In [9]:
df_fla = pd.read_csv('openaddress-bevlg.csv')
df_bru = pd.read_csv('openaddress-bebru.csv')
df_wal = pd.read_csv('openaddress-bewal.csv')

C:\Users\sijal\anaconda3\envs\3d_house\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (5,8,9,12,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sijal\anaconda3\envs\3d_house\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sijal\anaconda3\envs\3d_house\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (5,8,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Function to get the Full Address
---
#### *** Important note ***: Walloonia and Brussels  address are mostly in French 'fr' and as for Flemish it's in Dutch 'nl'
---
* convert postcode integer to string.
* combine streetname, house no. postcode, municipality name
* getting cordination and XY values 
* drop any dublicates
* Filter any row where XY value is zero

In [10]:
def get_address(df,lan='fr'):
    indiv_df = df
    indiv_df['postcode'] = indiv_df['postcode'].astype('str')
    indiv_df["Address"] = indiv_df[f"streetname_{lan}"] + " " + indiv_df["house_number"] + " " + indiv_df['postcode'] + " " +  indiv_df[f'municipality_name_{lan}']
    indiv_df_address =  indiv_df[['Address', 'EPSG:31370_x','EPSG:31370_y', 'EPSG:4326_lat','EPSG:4326_lon','region_code']]
    indiv_df_address.drop_duplicates(subset='Address', keep="first", inplace=True)
    filter_loc = (indiv_df_address['EPSG:31370_x'] != 0.0000) | (indiv_df_address['EPSG:31370_y'] != 0.0000)
    indiv_df_address = indiv_df_address[filter_loc]
    indiv_df_address.reset_index(drop=True,inplace=True)
    return indiv_df_address

# Individual Datafram of every regions 

In [11]:
df_fla_address = get_address(df_fla,"nl")
df_bru_address = get_address(df_bru)
df_wal_address = get_address(df_wal)

<ipython-input-10-e1771fc87ee8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_df_address.drop_duplicates(subset='Address', keep="first", inplace=True)
<ipython-input-10-e1771fc87ee8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_df_address.drop_duplicates(subset='Address', keep="first", inplace=True)
<ipython-input-10-e1771fc87ee8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indiv_df_address.drop_duplicates(subse

# Building a single dataframe of every region address and generate CSV file

In [12]:
df_BE_address = pd.concat([df_fla_address , df_bru_address , df_wal_address],ignore_index=True)
df_BE_address.head()
df_BE_address.to_csv('Belgium_Address.csv')

In [13]:
df_BE_address.head()

,Address,EPSG:31370_x,EPSG:31370_y,EPSG:4326_lat,EPSG:4326_lon,region_code
0,Steenweg op Oosthoven 51 2300 Turnhout,190710.56,224659.47,51.330292,4.952837,BE-VLG
1,Duivenstraat 102 2800 Mechelen,158832.72,192858.98,51.045828,4.494708,BE-VLG
2,Meersstraat 7 9185 Wachtebeke,114333.83,206615.16,51.168450,3.858827,BE-VLG
3,Nijverheidsstraat 67 9890 Gavere,101669.12,179169.21,50.920830,3.681393,BE-VLG
4,Klaverstraat 1A 9000 Gent,103887.16,195997.07,51.072272,3.710818,BE-VLG


## Searching for an address
---
* Print Longitute, Latitute, XY value of address if avaliable

In [18]:
search_adrs = input("Street name, House Number, Postcode, Muncipality \n\nEnter address :")

print("\n Processing!!!!! \n")

# get the index number of the search address
index = []

for i in range(df_BE_address.shape[0]):

    if df_BE_address['Address'][i].lower() == search_adrs.lower():
        index.append(i)
        break
    continue

try:
    x_y_value = df_BE_address['EPSG:31370_x'][index][index[0]] ,df_BE_address['EPSG:31370_y'][index][index[0]] 
    lat_lon = df_BE_address['EPSG:4326_lat'][index][index[0]] ,df_BE_address['EPSG:4326_lon'][index][index[0]] 

    print(f"EPSG:4326 (Geo Format) \nLatitude: {lat_lon[0]} \nLongitude: {lat_lon[1]}")
    print()
    print(f"EPSG:31370 (Bel Format) \nx value: {x_y_value[0]} \ny value: {x_y_value[1]}")

except:
    print("Unexpected Error:  Adress not found!!  Please provide valid Address")

Street name, House Number, Postcode, Muncipality 

Enter address :Bosuil 130 2100 Antwerpen

 Processing!!!!! 

EPSG:4326 (Geo Format) 
Latitude: 51.22773206349728 
Longitude: 4.465047290975518

EPSG:31370 (Bel Format) 
x value: 156726.29 
y value: 213092.83


In [10]:
import pandas as pd 
import folium

## Function to create Map Location using CSV - - (Optional)
---
### Alternative for the Geopy method

In [11]:
def csv_coordinate(search_address):
    
    index = []
    
    df = pd.read_csv(r"./address data/Belgium_Address.csv")
    
    for i in range(len(df)):

        if df['Address'][i] == search_address :
            index.append(i)
            break
        continue

    x, y = df['EPSG:31370_x'][index][index[0]] ,df['EPSG:31370_y'][index][index[0]] 

    lat, lon = df['EPSG:4326_lat'][index][index[0]] ,df['EPSG:4326_lon'][index][index[0]] 
    
    return (x, y) , (lat, lon)

def house_locate(func):
    
    # to plot address 

    house_locate = folium.Map(location=func,zoom_start=18)
    folium.Marker(location=func,
                  popup=list(func), 
                  icon=folium.Icon(color='green', icon='location-arrow', prefix='fa')  # Customize Icon
                 ).add_to(house_locate)
    #house_locate.save(address+".html")
    return house_locate

In [12]:
search_address = "Bosuil 130 2100 Antwerpen"

(x, y) , (lat, lon) = csv_coordinate(search_address)
print((x, y) , (lat, lon))
house_locate((lat, lon))

(156726.29, 213092.83) (51.22773206349728, 4.465047290975518)
